In [ ]:
import numpy as np
import matplotlib.cm
import matplotlib.pyplot as plt

import sys
sys.path.append('../')
import rtl_sdr_pr

In [ ]:
%%time

c = 3e8
cpi = 0.1 # in seconds
Fs = 2.88e6
fc = 625e6
num_samples = int(cpi * Fs)

with open('../data/gqrx_20210326_161137_625000000_2880000_fc_flugplatz_longdipole.raw', 'rb') as fid:
    n = rtl_sdr_pr.ioutils.read_samples(fid, num_samples)

step_distance = int(50e3)
num_iter = 1
step_samples = int(step_distance / c * Fs)
max_speed = 280 # m/s
max_doppler = int(max_speed * cpi * fc / c)
amb = np.empty((step_samples + 1, max_doppler, num_iter + 1), dtype=np.complex64)

for idx_win in range(num_iter):
    delay_bins = range(idx_win * step_samples, (idx_win + 1) * step_samples + 1)
    doppler_bins = np.arange(-max_doppler, max_doppler)
    amb = rtl_sdr_pr.processing.direct_ambiguity(delay_bins, doppler_bins, n, n, Fs)

In [ ]:
fig = plt.figure(figsize=(20, 5))
ax1 = fig.add_subplot(111)
ax1.set_xlabel("bistatic range [km]")
ax1.set_ylabel("correlation [dB]")
ax1_xticks = np.linspace(0, num_iter * step_distance / 1000, 22)
ax1.set_xticks(ax1_xticks)
ax1.plot(np.fromiter(delay_bins, dtype=float) / Fs * c / 1000, 10 * np.log10(np.abs(amb)))

ax2 = ax1.twiny()
ax2_xticks = np.linspace(0, num_iter * step_distance / 1000, 22)
ax2.set_xticks(ax2_xticks)
ax2.set_xticklabels(map(lambda tick: f"{(tick * 1e9) / 3e8:.0f}", ax2_xticks))
ax2.set_xlim(ax1.get_xlim())
ax2.set_xlabel("time [µsec]")
plt.title(f"Correlation of {cpi}sec CPI, Doppler: {1}", pad=40);

ax1.grid()
plt.grid(axis='x', alpha=0.5, linestyle='--')


In [ ]:
X, Y = np.meshgrid(np.arange(len(delay_bins)), np.arange(len(doppler_bins)))
Z = 10 * np.log10(np.abs(amb))[X, Y]

fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(10, 10))
ax.plot_surface(X, Y, Z, cmap=matplotlib.cm.Spectral)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.set_title("Range/Doppler Map")
ax.set_xlabel("bistatic range [km]")
ax.set_ylabel("bistatic velocity [m/s]")
xticks = np.linspace(0, 50e3, 6, endpoint=True)
ax.set_xticks(xticks / c * Fs)
ax.set_xticklabels(map(lambda x: f"{x // 1e3:.0f} km", xticks))

yticks = np.linspace(-max_speed, max_speed, 15, endpoint=True)
ax.set_yticks((yticks + max_speed) * cpi * fc / c)
ax.set_yticklabels(map(lambda y: f"{y:.0f} m/s", yticks))
plt.imshow(10 * np.log10(np.abs(amb.T)));